In [ ]:
!pip install langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install faiss-cpu
!pip install -U --quiet langchain-google-genai pillow

In [ ]:
import os
from langchain.prompts import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/Shareddrives/002_シラバスRAGシステム/data/shirabasu_html_Japanese_preprocessed.txt'
with open(path) as f:
    texts = f.readlines()
    texts = [x.replace('\n', '') for x in texts]

In [ ]:
embedding = HuggingFaceEmbeddings(model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite")
vectorstore = FAISS.from_texts(texts, embedding)
#埋め込みには1時間程かかります

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
<ipython-input-10-059c9d25ea51>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding = HuggingFaceEmbeddings(model_name = "oshizo/sbert-jsnli-luke-japanese-base-lite")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force consol

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.00k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

entity_vocab.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectorstore.save_local("./vectorstore")
#vectorestoreを保存

In [ ]:
vectorstore = FAISS.load_local("./vectorstore", embedding,allow_dangerous_deserialization=True)
#vectorestoreを読み込み

In [ ]:
retriever = vectorstore.as_retriever()
#retrieverの指定　デフォルトは類似度検索

template = """
ユーザー:Questionに当てはまる科目を複数個挙げて、それぞれの科目の概要を説明してください。

{context}

{question}

システム:"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"],
    template_format="f-string"
)
#プロンプトの作成

os.environ['GOOGLE_API_KEY'] = '******************************'
#https://ai.google.dev/からAPIのキーを取得
llm = ChatGoogleGenerativeAI(model='gemini-pro')
qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt},
    verbose=True,
)

q = "冷戦について学べる科目は何ですか？"
ans = qa.invoke(q)

print(ans['result'])

docs = ans['source_documents']
for d in docs:
    print()
    print(d.page_content)


> Finished chain.
- 国際政治論II

科目ナンバリングはU-LAS06 20021 LJ42。科目名は国際政治論II。英訳はInternational Politics II。所属部局、職 名、氏 名は人間・環境学研究科教授齋藤嘉臣。使用言語は日本語。単位数は2単位。授業形態は講義。開講年度・開講期は2024・後期。配当学年は全回生。対象学生は全学向。曜時限は水2。授業の概要・目的はポスト冷戦の国際安全保障体制はいかなる性質を持っているのだろうか。それは冷戦終焉以前の国際安全保障体制のいかなる基盤の上に成立しており、両者の間にはどのような相違があるのだろうか。本講義は、このような問題意識を念頭に、今日の国際社会が抱える安全保障上の様々な課題を検討することを通して、多様化する安全保障の視座を考察する。平和や安全保障を捉える視点は歴史的に変容しており、安全保障に関する焦点は、国家から国際機関さらには人間個人へと移行してきたとされる。安全保障上の脅威とされるものや、その脅威を取り除く手段についても、軍事的なものから非軍事的なものまで多様なものが想定されている。これら安全保障上の視座は、ウクライナ戦争の勃発によっても基本的には不変である。本講義は、今日の平和・安全保障の考え方を体系的に捉えることで、平和・安全保障についての概念的整理を行うとともに、それらがいかに現実の政策に反映されているか理解することを目的としている。到達目標は今日の安全保障問題を考察する際に重要となる、基本的な概念について十分な理解を得る。授業計画と内容は以下の内容で各１&#12316;２回、講義する。（授業回数はフィードバックを含め全15回とする）１安全保障とは何か２国家安全保障と国際安全保障３冷戦後の新しい安全保障論４グローバル化と新しい戦争５「自決」の主体は誰か６国際連合と平和構築７安全保障主体としてのEU・NATO８人間の安全保障とは何か９人道的介入・保護する責任とは何か１０武力行使に正義はあるか１１テロリズムとは何か。履修要件は特になし。成績評価の方法・観点は授業期間中に数回出すレポート課題による評価（１００％）３回以上、無断欠席した履修者には単位は与えない。教科書は使用しない。参考書等は授業中に紹介する。授業外学修（予習・復習）等は授業後には、事前に指定された参考書等の該